# Multi Layer Perceptron

### Overview
In this segment, we’ll build an intuitive understanding of what a **Multi-Layer Perceptron (MLP)** is and why it forms the foundation of modern deep learning. You’ll learn how a neuron works mathematically, how layers are stacked to form networks, and how activations introduce non-linearity to help models learn complex patterns.

### Agenda
- **What is an MLP?** → basic structure and intuition behind layered neural networks.  
- **How a Neuron Works** → weighted sum, bias, and activation function.  
- **Limitations of Linear Models** → why we need hidden layers and non-linear activations.  
- **MLP Architecture for MNIST** → understanding input, hidden, and output layers.  
- **Mathematical Representation** → equations for forward computations in an MLP.  

### Key Takeaway
By the end of this segment, you’ll understand how an MLP processes data, the role of each layer, and how activations enable it to capture non-linear patterns — setting the stage for building your own MLP model.

# Introduction to MLPs

It’s a **multi-layer perceptron model** created with `nn.Module`.

- **First layer (`nn.Linear`)**:  
  - Connects the **784 input features** (28×28 pixels from each MNIST image) to a **hidden layer** of neurons.  
  - Each hidden neuron learns its own weights and bias.  

- **Activation function (`ReLU`)**:  
  - Applied after the first layer.  
  - Keeps only positive values and sets negatives to zero, adding non-linearity.  

- **Second layer (`nn.Linear`)**:  
  - Connects the **hidden layer** to the **10 output classes** (digits 0–9).  
  - Produces one raw score (logit) for each digit.  

- **Forward pass**:  
  - Data flows through the input layer → hidden layer → output layer.  
  - Computation:  

    $h = \text{ReLU}(x W_{1}^{\!\top} + b_{1}), \quad y = h W_{2}^{\!\top} + b_{2}$

- **Application to MNIST**:  
  - Each image is flattened into **784 input features**.  
  - The model outputs **10 scores**, one for each digit.  
  - Using `softmax`, these scores are converted into probabilities.  
  - The digit with the highest probability is taken as the **predicted class**.  
  - Predictions are compared with the **true labels** from the test set to calculate **accuracy**.  
  - The code also prints **true vs predicted labels** for sample test images.

#### Importing Required Libraries ####

- **`import torch`** → loads the core PyTorch library for creating tensors and running computations on CPU/GPU.  
- **`import torch.nn as nn`** → imports PyTorch’s neural network module, which provides building blocks like layers (`Linear`, `Conv2d`, etc.).  
- **`import torch.nn.functional as F`** → imports a collection of functions (e.g., activation functions like `ReLU`, `softmax`, loss functions) that can be used directly.
- **`import torch.optim as optim`** → loads PyTorch’s optimisation module. Provides algorithms like **SGD** and **Adam** to update model weights during training.  
- **`import pickle`** → used to load and save Python objects; in this context, it helps load the MNIST dataset stored in a `.pkl` file.  

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle
from torch.utils.data import Dataset, DataLoader

#### Defining a Multi-Layer Perceptron (MLP)

The `MLP` class inherits from `nn.Module` and defines a simple feedforward network with two fully connected layers:  
- `fc1` maps inputs to a hidden layer and applies ReLU activation.  
- `fc2` maps the hidden layer to the outputs (logits).  

In the forward pass, data flows through `fc1 → ReLU → fc2`, producing the final output scores.

In [2]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        # Two fully connected layers
        self.fc1 = nn.Linear(input_size, hidden_size)  # input → hidden
        self.fc2 = nn.Linear(hidden_size, output_size) # hidden → output
    
    def forward(self, x):
        x = F.relu(self.fc1(x))   # first layer + ReLU
        x = self.fc2(x)           # second layer (logits)
        return x

#### Loading the MNIST Data
The MNIST dataset is loaded from a pickle file and split into training, validation, and test sets.  
- From this, we use the **training set** (`train_x`, `train_y`) and the **test set** (`test_x`, `test_y`).  
- Images are converted into `float32` tensors (`x_train`, `x_test`) for model input.  
- Labels are converted into `long` tensors (`y_train`, `y_test`) for classification tasks.  


In [3]:
class CustomMNISTDataset(Dataset):

    # defining constructor
    def __init__(self, path, tag="train", flatten=True):

        if tag == "train":
            images, labels = pickle.load(open(path, "rb"), encoding="latin1")[0]  # load train set
            
        elif tag == "test":
            images, labels = pickle.load(open(path, "rb"), encoding="latin1")[2]  # load test set
        
        x = torch.tensor(images, dtype=torch.float32)  # convert images to float32 tensor
        y = torch.tensor(labels, dtype=torch.long)     # convert labels to int64 tensor

        if flatten:                                    # flatten 28x28 → 784
            x = x.view(x.shape[0], -1)

        # set the class/dataset attributes
        self.x, self.y = x, y  # store tensors and optional transform

    # defining what is the length
    def __len__(self): 
        return len(self.y)                             # return total number of samples

    # defining how to get idx item
    def __getitem__(self, idx):
        img = self.x[idx]                              # get one image
        
        return img, self.y[idx]                        # return image and label


train_dataset = CustomMNISTDataset("data/mnist.pkl", tag="train", flatten=True)
test_dataset = CustomMNISTDataset("data/mnist.pkl", tag="test", flatten=True)


trainloader = DataLoader(train_dataset, batch_size=32)
testloader = DataLoader(test_dataset, batch_size=32)

#### Model, Loss, and Optimiser Setup

- We set the model dimensions with an input size of 784 (28×28 pixels), a hidden layer of 128 neurons, and an output size of 10 for the digit classes 0–9. 
- For training, we use **CrossEntropyLoss** to measure classification error and the **SGD optimiser** with a learning rate of 0.1 to update the model’s parameters.  


In [4]:
# Model, loss, optimizer
input_size = 784   # 28x28 pixels
hidden_size = 128  # hidden neurons
output_size = 10   # 10 digit classes

model_mlp = MLP(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()          # loss function
optimizer = optim.SGD(model_mlp.parameters(), lr=0.1)  # optimizer

### Training Loop

- Train the model for 5 epochs with mini-batches of size 64.  
- For each batch:  
  - Pass inputs (`xb`) through the model to get predictions.
  - Compare predictions with `yb` using the loss function. 
  - Compute the loss using the criterion.  
  - Perform backpropagation: reset gradients, compute new gradients, and update weights with the optimiser.  
- After each epoch, print the loss to monitor training progress.  


In [7]:
# Training loop
epochs = 10

for epoch in range(epochs):
    for input_, expected_output_ in trainloader:

        # Forward pass
        model_output_ = model_mlp(input_)

        # Compute loss
        loss = criterion(model_output_, expected_output_)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()

        # Update weights
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 1/10, Loss: 0.0279
Epoch 2/10, Loss: 0.0196
Epoch 3/10, Loss: 0.0138
Epoch 4/10, Loss: 0.0093
Epoch 5/10, Loss: 0.0069
Epoch 6/10, Loss: 0.0057
Epoch 7/10, Loss: 0.0047
Epoch 8/10, Loss: 0.0040
Epoch 9/10, Loss: 0.0029
Epoch 10/10, Loss: 0.0026


### Evaluation on the Test Set

- Switching to evaluation mode with `torch.no_grad()` to disable gradient tracking.  
- Passing the test data (`x_test`) through the model to get output scores.  
- Applying **softmax** and use **argmax** to select the predicted class for each sample.  
- Comparing predictions with true labels (`y_test`) to count correct matches and calculate accuracy.  
- Printing the final test accuracy as a percentage, and also showing a few sample predictions (first 10) with their true labels to see how the model performs on individual cases.  


In [8]:
# Evaluation on test set
test_input_ = test_dataset[:][0]
test_expected_output = test_dataset[:][1]

with torch.no_grad():
    model_output_ = model_mlp(test_input_)
    model_class_ = torch.argmax(F.softmax(model_output_, dim=0), dim=1)

    correct = (model_class_ == test_expected_output).sum().item()
    accuracy = correct / len(test_input_)

print(f"\nTest Accuracy: {accuracy*100:.2f}%")

# Show some sample predictions
print("\nSample Predictions (first 10):")
for i in range(10):
    print(f"True: {test_expected_output[i].item()}  Predicted: {model_class_[i].item()}")


Test Accuracy: 84.66%

Sample Predictions (first 10):
True: 7  Predicted: 7
True: 2  Predicted: 1
True: 1  Predicted: 1
True: 0  Predicted: 0
True: 4  Predicted: 4
True: 1  Predicted: 1
True: 4  Predicted: 4
True: 9  Predicted: 9
True: 5  Predicted: 6
True: 9  Predicted: 9


#### Conclusion
  - Extends the Neuron into a two-layer network with a hidden layer and ReLU activation.  
  - Capable of learning more complex patterns when trained.  
  - In practice, after training, it achieves much higher accuracy on MNIST (96%).  
  - Demonstrates how adding depth and non-linearity improves performance.
